In [1]:
# Imports
import os
import pandas as pd
import numpy as np

print("Setup Complete")

Setup Complete


In [2]:
# Read csv
file = "_input_payroll_list/payroll_list_sample.csv"
df_salary = pd.read_csv(file)
df_salary


,name,net_salary,net_salary_int
0,person1,4213.84,4213
1,person2,8133.18,8133
2,person3,14078.21,14078
3,person4,16192.09,16192
4,person5,10575.76,10575
5,person6,10016.63,10016
6,person7,8296.99,8296
7,person8,13118.18,13118
8,person9,8535.31,8535
9,person10,10695.44,10695


In [3]:
# Define a function to perform a Knapsack Unbounded Algorithm
def create_denomination_breakdown(
    amounts: list, total_denominations: dict, denominations: list
) -> dict:
    """
    Calculate the denomination breakdowns for a given set of amounts and available denominations.
    
    Parameters:
    - amounts (list): List of amounts to calculate the breakdowns for.
    - total_denominations (dict): Dictionary representing the available denominations and their counts.
    - denominations (list): List of denominations to consider.
    
    Returns:
    - breakdowns (dict): List of breakdown dictionaries, each representing the denomination breakdown for a given amount.
    """

    breakdowns = []
    num_amounts = len(amounts)

    def unbounded_knapsack(amount, denominations, total_denominations):
        dp = [0] + [float("inf")] * amount
        prev = [-1] * (amount + 1)

        for denom in denominations:
            if total_denominations[denom] > 0:  # Check if denomination is available
                for j in range(denom, amount + 1):
                    if dp[j - denom] + 1 < dp[j]:
                        dp[j] = dp[j - denom] + 1
                        prev[j] = denom

        return prev

    for i in range(num_amounts):
        amount = amounts[i]
        breakdown = {}

        prev = unbounded_knapsack(amount, denominations, total_denominations)

        while amount > 0:
            denom = prev[amount]
            breakdown[denom] = breakdown.get(denom, 0) + 1
            amount -= denom
            total_denominations[
                denom
            ] -= 1  # Decrement the available denomination count

        breakdowns.append(breakdown)

    return breakdowns


# Define a function to create a Dataframe
def create_df(
    denom_breakdowns: dict, amounts: list, names: list, denominations: list
) -> pd.DataFrame:
    """
    Create a DataFrame from the denomination breakdowns, amounts, names, and denominations.
    
    Parameters:
    - denom_breakdowns (dict): List of breakdown dictionaries representing the denomination breakdowns for each amount.
    - amounts (list): List of amounts.
    - names (list): List of names corresponding to the amounts.
    - denominations (list): List of denominations.
    
    Returns:
    - df (pd.DataFrame): DataFrame containing the breakdown information.
    """
    
    data = []
    for i, breakdown in enumerate(denom_breakdowns):
        row = {f"P{d}x": breakdown.get(d, 0) for d in denominations}
        row["amount"] = amounts[i]
        row["name"] = names[i]
        row["net_salary_int"] = int(
            round(amounts[i])
        )  # Round the amount and convert to int
        row["variance"] = (
            sum(d * row.get(f"P{d}x", 0) for d in denominations) - row["net_salary_int"]
        )
        data.append(row)
    df = pd.DataFrame(data)
    columns_order = (
        ["name", "net_salary_int"] + [f"P{d}x" for d in denominations] + ["variance"]
    )
    df = df[columns_order]
    return df


# Parameters
amounts = df_salary["net_salary_int"].to_list()
names = df_salary["name"].to_list()
total_denominations = {1_000: 177, 
                       500: 100, 
                       100: 102, 
                       50: 100, 
                       20: 100, 
                       5: 10, 
                       1: 90
                    }

denominations = [1_000, 500, 100, 50, 20, 5, 1]

# Create denomination breakdowns
denomination_breakdowns = create_denomination_breakdown(
    amounts, total_denominations, denominations
)

# Create Dataframe
df_denom = create_df(denomination_breakdowns, amounts, names, denominations)
df_denom


,name,net_salary_int,P1000x,P500x,P100x,P50x,P20x,P5x,P1x,variance
0,person1,4213,4,0,2,0,0,2,3,0
1,person2,8133,8,0,1,0,1,2,3,0
2,person3,14078,14,0,0,1,1,1,3,0
3,person4,16192,16,0,1,1,2,0,2,0
4,person5,10575,10,1,0,1,1,1,0,0
5,person6,10016,10,0,0,0,0,3,1,0
6,person7,8296,8,0,2,1,2,1,1,0
7,person8,13118,13,0,0,1,3,0,8,0
8,person9,8535,8,0,4,1,4,0,5,0
9,person10,10695,10,1,1,1,2,0,5,0


In [4]:
# Save as csv file
output_file = "_output_payroll_list/output_payroll_list_sample.csv"
df_denom.drop(columns=["variance"]).to_csv(output_file, index=True)

In [5]:
# Check computed denominations
print(f"1000x {df_denom['P1000x'].sum()}")
print(f"500x {df_denom['P500x'].sum()}")
print(f"100x {df_denom['P100x'].sum()}")
print(f"50x {df_denom['P50x'].sum()}")
print(f"20x {df_denom['P20x'].sum()}")
print(f"5x {df_denom['P5x'].sum()}")
print(f"1x {df_denom['P1x'].sum()}")
print(f"net_salary_int {df_denom['net_salary_int'].sum():,}")


1000x 180
500x 100
100x 125
50x 15
20x 42
5x 10
1x 81
net_salary_int 244,221
